In [1]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent))
sys.path.append(str(Path.cwd().parent / 'label_anything'))
sys.path.append(str(Path.cwd().parent / 'label_anything' / 'data'))

In [2]:
import torch
from PIL import Image
from torchvision.transforms import Compose, PILToTensor
from label_anything.data.transforms import CustomNormalize, CustomResize
from label_anything.data.dataset import LabelAnythingDataset
import numpy as np
import matplotlib.pyplot as plt
from label_anything.logger.text_logger import get_logger
from label_anything.logger.utils import (
    extract_boxes_from_tensor,
    image_with_points,
    structure_annotations
)
import comet_ml
from label_anything.logger.image_logger import Logger
from torch.utils.data import DataLoader
import cv2
from torchvision.transforms.functional import resize

/home/nico/Development/LabelAnything/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'extract_vertices_from_tensor' from 'label_anything.logger.utils' (/home/nico/Development/LabelAnything/label_anything/logger/utils.py)

In [ ]:
RAW_DATA_DIR = Path.cwd().parent / "data" / "raw"

In [ ]:
comet_ml.init(project_name='label-anything')

In [ ]:
experiment = comet_ml.Experiment()
logger = Logger(experiment)

In [ ]:
preprocess = Compose([
    CustomResize(1024),
    PILToTensor(),
    CustomNormalize(1024)
])

In [ ]:
dataset = LabelAnythingDataset(
    instances_path=RAW_DATA_DIR / "annotations" / "instances_train2017.json",
    img_dir=RAW_DATA_DIR / "train2017",
    preprocess=preprocess
)

In [ ]:
dataloader = DataLoader(
    dataset=dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=dataset.collate_fn
)

In [ ]:
batch, gt = next(iter(dataloader))

In [ ]:
logger.log_batch(0, 0, batch, dataset.categories)

In [ ]:
def get_image(image_tensor):
    MEAN = np.array([123.675, 116.280, 103.530]) / 255
    STD = np.array([58.395, 57.120, 57.375]) / 255
    unnormalized_image = (image_tensor.numpy() * np.array(STD)[:, None, None]) + np.array(MEAN)[:, None, None]
    unnormalized_image = (unnormalized_image * 255).astype(np.uint8)
    unnormalized_image = np.moveaxis(unnormalized_image, 0, -1)
    return Image.fromarray(unnormalized_image)

In [ ]:
mask = batch["prompt_masks"][0, 2, 1]

In [ ]:
mask.unsqueeze(0).shape

In [ ]:
mask = resize(mask.unsqueeze(0), (1024, 1024), interpolation=Image.NEAREST)
mask = np.array(mask).astype(np.uint8)

In [ ]:
mask = mask.squeeze() 

In [ ]:
contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
polygons = []

for obj in contours:
    coords = []
        
    for point in obj:
        coords.append(int(point[0][0]))
        coords.append(int(point[0][1]))

    polygons.append(coords)

In [ ]:
polygons

In [ ]:
image = batch["images"][0, 2]
mask = batch["prompt_masks"][0, 2, 1]
annotations_mask = structure_annotations(
    polygons,
)
logger.log_image(
    get_image(image),
    annotations_mask,
)